# Getting Started Kaggle TPS Challenge with Tabular ML Toolkit

> A Tutorial to showcase usage of tabular_ml_toolkit (tmlt) library on Kaggle TPS Challenge Nov 2021.

> tabular_ml_toolkit is a helper library to jumpstart your machine learning project based on Tabular or Structured data.

> It comes with model and data parallelism and cutting edge hyperparameter search techniques.

> Under the hood TMLT uses modin, optuna, xgboost and scikit-learn pipelines

## Install

`pip install -U tabular_ml_toolkit`

### How to Best Use tabular_ml_toolkit

Start with your favorite model and then just simply create **tmlt** with one API.

*Here we are using XGBClassifier, on  [Kaggle TPS Challenge (Nov 2021) data](https://www.kaggle.com/c/tabular-playground-series-nov-2021/data)*

In [ ]:
from tabular_ml_toolkit.tmlt import *
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
import numpy as np

# for visualizing pipeline
from sklearn import set_config
set_config(display="diagram")

# just to measure fit performance
import time

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
# Dataset file names and Paths
DIRECTORY_PATH = "/Users/pamathur/kaggle_datasets/tps_nov_2021/"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"
OUTPUT_PATH = "kaggle_tps_output/"

In [ ]:
# TRY THIS using LOGISTIC Regression
# https://www.kaggle.com/maximkazantsev/tps-11-21-eda-xgboost-optuna

# ALSO TAKE OUT MODIN OR USE SOME FUNCTIONALITY TO USE BOTH

#### Create a base xgb classifier model with your best guess params

In [ ]:
xgb_params = {
    # your best guess params
    'learning_rate':0.01,
    'eval_metric':'auc',
    # must for xgb classifier otherwise warning will be shown
    'use_label_encoder':False,
    # because 42 is the answer for all the randomness of this universe
    'random_state':42,
    #for GPU
    #'tree_method': 'gpu_hist',
    #'predictor': 'gpu_predictor',
}

xgb_model = XGBClassifier(**xgb_params)

In [ ]:
# createm tmlt for xgb model
tmlt = TMLT().prepare_data_for_training(
    train_file_path= DIRECTORY_PATH + TRAIN_FILE,
    test_file_path= DIRECTORY_PATH + TEST_FILE,
    #make sure to use right index and target columns
    idx_col="id",
    target="target",
    model=xgb_model,
    random_state=42,
    problem_type="binary_classification", nrows=4000)


# supports only task type
# "binary_classification"
# "multi_label_classification"
# "multi_class_classification"
# "regression"

2021-11-24 11:53:15,358 INFO 12 cores found, model and data parallel processing should worked!
2021-11-24 11:53:15,477 INFO DataFrame Memory usage decreased to 0.80 Mb (74.4% reduction)
2021-11-24 11:53:15,601 INFO DataFrame Memory usage decreased to 0.79 Mb (74.3% reduction)
2021-11-24 11:53:15,655 INFO categorical columns are None, Preprocessing will done accordingly!


In [ ]:
tmlt.spl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['f0', 'f1', 'f2', 'f3', 'f4',
                                                   'f5', 'f6', 'f7', 'f8', 'f9',
                                                   'f10', 'f11', 'f12', 'f13',
                                                   'f14', 'f15', 'f16', 'f17',
                                                   'f18', 'f19', 'f20', 'f21',
                                                   'f22', 'f23', 'f24', 'f25',
                                                   'f26', 'f27', 'f28', 'f29', ...])])),
                (...
                               interaction_constraints=None, learning_rate=0.01,
                               max_delta_step=None, max_depth=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=11, num_parallel_tree=None,
                               predictor=None, random_state=42, reg_alpha=None,
                               reg_lambda=None, scale_pos_weight=None,
                               subsample=None, tree_method=None,
                               use_label_encoder=False,
                               validate_parameters=None, verbosity=None))])

#### Let's do a quick round of training

In [ ]:
tmlt.dfl.create_train_valid(valid_size=0.2)

In [ ]:
# Quick check on dataframe shapes
print(f"X_train shape is {tmlt.dfl.X_train.shape}" )
print(f"X_valid shape is {tmlt.dfl.X_valid.shape}" )
print(f"y_train shape is {tmlt.dfl.y_train.shape}")
print(f"y_valid shape is {tmlt.dfl.y_valid.shape}")

X_train shape is (3200, 100)
X_valid shape is (800, 100)
y_train shape is (3200,)
y_valid shape is (800,)


In [ ]:
# Fit
start = time.time()
# Now fit
tmlt.spl.fit(tmlt.dfl.X_train, tmlt.dfl.y_train)
end = time.time()
print("Fit Time:", end - start)

#predict
preds = tmlt.spl.predict(tmlt.dfl.X_valid)
preds_probs = tmlt.spl.predict_proba(tmlt.dfl.X_valid)[:, 1]

# Metrics
auc = roc_auc_score(tmlt.dfl.y_valid, preds_probs)
acc = accuracy_score(tmlt.dfl.y_valid, preds)

print(f"AUC is : {auc} while Accuracy is : {acc} ")

Fit Time: 1.045884132385254
AUC is : 0.6137947418435223 while Accuracy is : 0.6175 


#### Base model For Meta Ensemble Model 

In [ ]:
# OOF training and prediction on both train and test dataset by a given model

linear_oof_model = LinearSVC(tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=42)

linear_oof_model_preds, linear_oof_model_test_preds = tmlt.do_oof_kfold_train_preds(n_splits=5,
                                                          oof_model=linear_oof_model)
if linear_oof_model_preds is not None:
    print(linear_oof_model_preds.shape)

if linear_oof_model_test_preds is not None:    
    print(linear_oof_model_test_preds.shape)

2021-11-24 11:55:33,649 INFO fold: 1 OOF Model ROC AUC: 0.7259767891682785!
2021-11-24 11:55:34,094 INFO fold: 2 OOF Model ROC AUC: 0.6958091553836234!
2021-11-24 11:55:34,543 INFO fold: 3 OOF Model ROC AUC: 0.6614764667956157!
2021-11-24 11:55:35,027 INFO fold: 4 OOF Model ROC AUC: 0.7080050760440353!
2021-11-24 11:55:35,446 INFO fold: 5 OOF Model ROC AUC: 0.7223571396363027!
2021-11-24 11:55:35,451 INFO Mean OOF Model ROC AUC: 0.7027249254055712!


(4000,)
(4000,)


In [ ]:
# add based model oof predictions back to X and X_test before Meta model training
tmlt.dfl.X["linear_preds"] = linear_oof_model_preds
tmlt.dfl.X_test["linear_preds"] = linear_oof_model_test_preds

In [ ]:
print(tmlt.dfl.X.shape)
print(tmlt.dfl.X_test.shape)

(4000, 101)
(4000, 101)


#### For Meta Model, Let's do Optuna based HyperParameter search to get best params for fit

In [ ]:
study = tmlt.do_xgb_optuna_optimization(optuna_db_path=OUTPUT_PATH, opt_timeout=360)

2021-11-24 11:56:05,611 INFO Optimization Direction is: minimize
[I 2021-11-24 11:56:05,674] Using an existing study with name 'tmlt_autoxgb' instead of creating a new one.
2021-11-24 11:56:05,948 INFO Training Started!


[11:56:05] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 11:56:13,976 INFO Training Ended!
2021-11-24 11:56:14,038 INFO log_loss: 0.6036267434991897
2021-11-24 11:56:14,039 INFO roc_auc_score: 0.7167207792207793
2021-11-24 11:56:14,039 INFO accuracy_score: 0.70125
2021-11-24 11:56:14,040 INFO f1_score: 0.5886402753872634
2021-11-24 11:56:14,041 INFO precision_score: 0.6263736263736264
2021-11-24 11:56:14,041 INFO recall_score: 0.5551948051948052
[I 2021-11-24 11:56:14,078] Trial 40 finished with value: 0.6036267434991897 and parameters: {'learning_rate': 0.0467094492253725, 'n_estimators': 15000, 'reg_lambda': 6.538273265404555e-05, 'reg_alpha': 0.0013117821568648178, 'subsample': 0.5290631340507302, 'colsample_bytree': 0.7582355779106297, 'max_depth': 4, 'early_stopping_rounds': 426, 'tree_method': 'exact', 'booster': 'gblinear'}. Best is trial 20 with value: 0.6023122700396926.
2021-11-24 11:56:14,250 INFO Training Started!


[11:56:14] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 11:56:17,358 INFO Training Ended!
/Users/pamathur/miniconda3/envs/nbdev_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2021-11-24 11:56:17,419 INFO log_loss: 0.6689949867129326
2021-11-24 11:56:17,420 INFO roc_auc_score: 0.5
2021-11-24 11:56:17,420 INFO accuracy_score: 0.615
2021-11-24 11:56:17,421 INFO f1_score: 0.0
2021-11-24 11:56:17,422 INFO precision_score: 0.0
2021-11-24 11:56:17,422 INFO recall_score: 0.0
[I 2021-11-24 11:56:17,451] Trial 41 finished with value: 0.6689949867129326 and parameters: {'learning_rate': 0.018704968140364132, 'n_estimators': 15000, 'reg_lambda': 3.5773118385391564e-06, 'reg_alpha': 0.3568074678915505, 'subsample': 0.5658420728617757, 'colsample_bytree': 0.6677211087049771, 'max_depth': 7, 'early_stopping_round

...

2021-11-24 12:01:47,788 INFO Training Ended!
2021-11-24 12:01:47,848 INFO log_loss: 0.6034830134734511
2021-11-24 12:01:47,849 INFO roc_auc_score: 0.7191888396156689
2021-11-24 12:01:47,850 INFO accuracy_score: 0.6975
2021-11-24 12:01:47,850 INFO f1_score: 0.5583941605839416
2021-11-24 12:01:47,851 INFO precision_score: 0.6375
2021-11-24 12:01:47,852 INFO recall_score: 0.4967532467532468
[I 2021-11-24 12:01:47,880] Trial 68 finished with value: 0.6034830134734511 and parameters: {'learning_rate': 0.24018034186076134, 'n_estimators': 15000, 'reg_lambda': 0.06341025893382701, 'reg_alpha': 2.2796329413542294e-08, 'subsample': 0.9599350543076591, 'colsample_bytree': 0.7355242470499104, 'max_depth': 7, 'early_stopping_rounds': 166, 'tree_method': 'exact', 'booster': 'gblinear'}. Best is trial 64 with value: 0.602295038830489.
2021-11-24 12:01:48,044 INFO Training Started!


[12:01:48] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 12:01:59,873 INFO Training Ended!
2021-11-24 12:01:59,942 INFO log_loss: 0.6022779347747564
2021-11-24 12:01:59,942 INFO roc_auc_score: 0.7178690212226798
2021-11-24 12:01:59,943 INFO accuracy_score: 0.69875
2021-11-24 12:01:59,944 INFO f1_score: 0.5734513274336284
2021-11-24 12:01:59,944 INFO precision_score: 0.6303501945525292
2021-11-24 12:01:59,945 INFO recall_score: 0.525974025974026
[I 2021-11-24 12:01:59,974] Trial 69 finished with value: 0.6022779347747564 and parameters: {'learning_rate': 0.152746085461703, 'n_estimators': 15000, 'reg_lambda': 0.031987234461410045, 'reg_alpha': 1.3693071141813216e-07, 'subsample': 0.8214620713008292, 'colsample_bytree': 0.8000514536230441, 'max_depth': 7, 'early_stopping_rounds': 147, 'tree_method': 'exact', 'booster': 'gblinear'}. Best is trial 69 with value: 0.6022779347747564.
2021-11-24 12:02:00,262 INFO Training Started!


[12:02:00] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 12:02:11,425 INFO Training Ended!
2021-11-24 12:02:11,485 INFO log_loss: 0.6026194911077618
2021-11-24 12:02:11,486 INFO roc_auc_score: 0.7173410938654842
2021-11-24 12:02:11,486 INFO accuracy_score: 0.69875
2021-11-24 12:02:11,487 INFO f1_score: 0.5823223570190641
2021-11-24 12:02:11,488 INFO precision_score: 0.6245353159851301
2021-11-24 12:02:11,488 INFO recall_score: 0.5454545454545454
[I 2021-11-24 12:02:11,514] Trial 70 finished with value: 0.6026194911077618 and parameters: {'learning_rate': 0.1630843141366229, 'n_estimators': 20000, 'reg_lambda': 0.018155430643457735, 'reg_alpha': 6.924580799000354e-08, 'subsample': 0.8331594340685916, 'colsample_bytree': 0.7983840960926235, 'max_depth': 7, 'early_stopping_rounds': 131, 'tree_method': 'hist', 'booster': 'gblinear'}. Best is trial 69 with value: 0.6022779347747564.


In [ ]:
print(study.best_trial)

FrozenTrial(number=69, values=[0.6022779347747564], datetime_start=datetime.datetime(2021, 11, 24, 12, 1, 47, 891300), datetime_complete=datetime.datetime(2021, 11, 24, 12, 1, 59, 946156), params={'booster': 'gblinear', 'colsample_bytree': 0.8000514536230441, 'early_stopping_rounds': 147, 'learning_rate': 0.152746085461703, 'max_depth': 7, 'n_estimators': 15000, 'reg_alpha': 1.3693071141813216e-07, 'reg_lambda': 0.031987234461410045, 'subsample': 0.8214620713008292, 'tree_method': 'exact'}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear')), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'early_stopping_rounds': IntUniformDistribution(high=500, low=100, step=1), 'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'n_estimators': CategoricalDistribution(choices=(7000, 15000, 20000)), 'reg_alpha': LogUniformDistribution(high=100.0, low=1e-08), 'reg_lambda': LogUniformDistribu

##### now update the meta model with best params from study and then update the sklearn pipeline with this new model

In [ ]:
xgb_params.update(study.best_trial.params)
print("Final xgb_params:", xgb_params)
xgb_model = XGBClassifier(**xgb_params)
tmlt.update_model(xgb_model)
tmlt.spl

Final xgb_params: {'learning_rate': 0.152746085461703, 'eval_metric': 'auc', 'use_label_encoder': False, 'random_state': 42, 'booster': 'gblinear', 'colsample_bytree': 0.8000514536230441, 'early_stopping_rounds': 147, 'max_depth': 7, 'n_estimators': 15000, 'reg_alpha': 1.3693071141813216e-07, 'reg_lambda': 0.031987234461410045, 'subsample': 0.8214620713008292, 'tree_method': 'exact'}


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['f0', 'f1', 'f2', 'f3', 'f4',
                                                   'f5', 'f6', 'f7', 'f8', 'f9',
                                                   'f10', 'f11', 'f12', 'f13',
                                                   'f14', 'f15', 'f16', 'f17',
                                                   'f18', 'f19', 'f20', 'f21',
                                                   'f22', 'f23', 'f24', 'f25',
                                                   'f26', 'f27', 'f28', 'f29', ...])])),
                (...
                               max_delta_step=None, max_depth=7,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=15000,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=42,
                               reg_alpha=1.3693071141813216e-07,
                               reg_lambda=0.031987234461410045,
                               scale_pos_weight=None,
                               subsample=0.8214620713008292,
                               tree_method='exact', use_label_encoder=False,
                               validate_parameters=None, ...))])

#### Let's Use K-Fold Training with best params

In [ ]:
# K-Fold fit and predict on test dataset
xgb_model_mean_metrics_results, xgb_model_test_preds= tmlt.do_kfold_training(n_splits=5,
                                                                            test_preds_metric=roc_auc_score)
if xgb_model_test_preds is not None:
    print(xgb_model_test_preds.shape)

[12:02:11] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "early_stopping_rounds", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 12:02:21,265 INFO fold: 1 log_loss : 0.6044875546731054
2021-11-24 12:02:21,266 INFO fold: 1 roc_auc_score : 0.7267569310122501
2021-11-24 12:02:21,266 INFO fold: 1 accuracy_score : 0.69125
2021-11-24 12:02:21,267 INFO fold: 1 f1_score : 0.5612788632326821
2021-11-24 12:02:21,267 INFO fold: 1 precision_score : 0.6781115879828327
2021-11-24 12:02:21,268 INFO fold: 1 recall_score : 0.47878787878787876
2021-11-24 12:02:21,269 INFO Predicting Test Preds Probablities!


[12:02:21] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "early_stopping_rounds", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 12:02:30,239 INFO fold: 2 log_loss : 0.6239040436223149
2021-11-24 12:02:30,240 INFO fold: 2 roc_auc_score : 0.6955383623468729
2021-11-24 12:02:30,240 INFO fold: 2 accuracy_score : 0.68875
2021-11-24 12:02:30,241 INFO fold: 2 f1_score : 0.5815126050420169
2021-11-24 12:02:30,242 INFO fold: 2 precision_score : 0.6528301886792452
2021-11-24 12:02:30,243 INFO fold: 2 recall_score : 0.5242424242424243
2021-11-24 12:02:30,243 INFO Predicting Test Preds Probablities!


[12:02:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "early_stopping_rounds", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




...

[12:02:39] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "early_stopping_rounds", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 12:02:49,262 INFO fold: 4 log_loss : 0.617499795970507
2021-11-24 12:02:49,262 INFO fold: 4 roc_auc_score : 0.7093964789775765
2021-11-24 12:02:49,263 INFO fold: 4 accuracy_score : 0.69375
2021-11-24 12:02:49,264 INFO fold: 4 f1_score : 0.5840407470288624
2021-11-24 12:02:49,264 INFO fold: 4 precision_score : 0.6666666666666666
2021-11-24 12:02:49,265 INFO fold: 4 recall_score : 0.5196374622356495
2021-11-24 12:02:49,265 INFO Predicting Test Preds Probablities!


[12:02:49] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/learner.cc:576: 
Parameters: { "colsample_bytree", "early_stopping_rounds", "max_depth", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




2021-11-24 12:02:58,122 INFO fold: 5 log_loss : 0.6016531882900744
2021-11-24 12:02:58,123 INFO fold: 5 roc_auc_score : 0.7217580633732503
2021-11-24 12:02:58,124 INFO fold: 5 accuracy_score : 0.6775
2021-11-24 12:02:58,124 INFO fold: 5 f1_score : 0.5582191780821918
2021-11-24 12:02:58,125 INFO fold: 5 precision_score : 0.6442687747035574
2021-11-24 12:02:58,125 INFO fold: 5 recall_score : 0.49244712990936557
2021-11-24 12:02:58,126 INFO Predicting Test Preds Probablities!
2021-11-24 12:02:58,155 INFO  Mean Metrics Results from all Folds are: {'log_loss': 0.6181551259793341, 'roc_auc_score': 0.7032599220098172, 'accuracy_score': 0.6795, 'f1_score': 0.5568332875267081, 'precision_score': 0.6483754436064604, 'recall_score': 0.48847752448960907}


(4000,)


In [ ]:
# # take weighted average of both k-fold models predictions
# final_preds = ((0.45 * sci_model_preds) + (0.55* xgb_model_test_preds)) / 2
# print(final_preds.shape)

#### Create Kaggle Predictions

In [ ]:
# sub = pd.read_csv(DIRECTORY_PATH + SAMPLE_SUB_FILE)
# sub['target'] = final_preds
# sub.to_csv('submission.csv', index=False)

In [ ]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()